#### 1. Download the shopify dataset:

In [4]:
%env BASE_DIR=/home/jupyter/

env: BASE_DIR=/home/jupyter/


In [5]:
!gsutil cp gs://cs327e-open-access-2/shopify.zip $BASE_DIR

Copying gs://cs327e-open-access-2/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [6]:
!unzip $BASE_DIR/shopify.zip

Archive:  /home/jupyter//shopify.zip
   creating: shopify/
  inflating: __MACOSX/._shopify      
  inflating: shopify/apps_categories.csv  
  inflating: __MACOSX/shopify/._apps_categories.csv  
  inflating: shopify/categories.csv  
  inflating: __MACOSX/shopify/._categories.csv  
  inflating: shopify/reviews.csv     
  inflating: __MACOSX/shopify/._reviews.csv  
  inflating: shopify/.DS_Store       
  inflating: __MACOSX/shopify/._.DS_Store  
  inflating: shopify/key_benefits.csv  
  inflating: __MACOSX/shopify/._key_benefits.csv  
  inflating: shopify/pricing_plan_features.csv  
  inflating: __MACOSX/shopify/._pricing_plan_features.csv  
  inflating: shopify/pricing_plans.csv  
  inflating: __MACOSX/shopify/._pricing_plans.csv  
  inflating: shopify/apps.csv        
  inflating: __MACOSX/shopify/._apps.csv  


#### 2. Consolidate apps_categories and categories & pricing_plans and pricing_plan_features

In [9]:
import pandas as pd
def merge_fun(left_df, right_df, left_key, right_key, howto):
    merged_df = pd.merge(left_df, right_df, left_on=left_key, right_on=right_key, how=howto)
    merged_df.drop(left_key,axis=1, inplace=True)
    #print(merged_df)
    return merged_df


# Load the CSV files into pandas DataFrames
csvLeft = pd.read_csv('/home/jupyter/shopify/pricing_plans.csv')
csvRight = pd.read_csv('/home/jupyter/shopify/pricing_plan_features.csv')
leftkey=['app_id','id']
rightkey=['app_id','pricing_plan_id']

merged_df=merge_fun(csvLeft, csvRight, leftkey, rightkey, 'left' )
#save new csv file
merged_df.to_csv('/home/jupyter/shopify/pricing_plan_details.csv', index=False)

csvLeft = pd.read_csv('/home/jupyter/shopify/apps_categories.csv')
csvRight = pd.read_csv('/home/jupyter/shopify/categories.csv')

merged_df=merge_fun(csvLeft, csvRight, 'category_id','id','inner')
#save new csv file
merged_df.to_csv('/home/jupyter/shopify/category.csv', index=False)


#### 3. Create and populate Firestore database

In [5]:
import pandas as pd
from google.cloud import firestore
from datetime import datetime

db = firestore.Client()
batch = db.batch()
df = pd.read_csv('/home/jupyter/shopify/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

for row in rows: 
    record = {}
    record['id'] = row[0]
    record['title'] = row[1]
    
    categories_ref = db.collection('categories').document(row[0])
    batch.set(categories_ref, record)
    
batch.commit()

[update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 },
 update_time {
   seconds: 1711668451
   nanos: 649219000
 }]

In [8]:
from google.cloud import firestore
db = firestore.Client()

def delete_subcollections(db, collection_ref, subcollection_name, batch_size):
    apps = collection_ref.stream()
    for app_doc in apps:
        subcollection_ref = app_doc.reference.collection(subcollection_name)
        docs = subcollection_ref.limit(batch_size).stream()
        deleted = 0
        batch = db.batch()
        for doc in docs:
            batch.delete(doc.reference)
            deleted += 1
            if deleted == batch_size:
                batch.commit()
                batch = db.batch()
                deleted = 0
        if batch:
            batch.commit()


collection_ref = db.collection('apps')
subcollection_name = 'reviews'  
delete_subcollections(db, collection_ref, subcollection_name, 10)  # Adjust batch_size as needed
subcollection_name = 'key_benefits'
delete_subcollections(db, collection_ref, subcollection_name, 10)  # Adjust batch_size as needed

print("Sub Deletion complete!")


# Delete documents in batches of 500
batch = db.batch()
num_mutations = 0
for doc in collection_ref.stream():
    batch.delete(doc.reference)
    num_mutations = num_mutations+1
    if num_mutations >= 500:
        batch.commit()
        batch = db.batch()

# Commit any remaining deletions
batch.commit() 
print("delete apps done")

Sub Deletion complete!
delete apps done


In [10]:
import pandas as pd
from google.cloud import firestore
from datetime import datetime
import uuid

db = firestore.Client()
batch = db.batch()

df = pd.read_csv('/home/jupyter/shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()
df = pd.read_csv('/home/jupyter/shopify/category.csv', sep=',', header=0, lineterminator='\n')
category_rows = df.values.tolist()
df = pd.read_csv('/home/jupyter/shopify/categories.csv', sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()
df = pd.read_csv('/home/jupyter/shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
review_rows = df.values.tolist()
review_count = len(review_rows)
df = pd.read_csv('/home/jupyter/shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
key_benefit_rows = df.values.tolist()
benefit_count=len(key_benefit_rows)
df = pd.read_csv('/home/jupyter/shopify/pricing_plan_details.csv', sep=',', header=0, lineterminator='\n')
pricing_plan_detail_rows = df.values.tolist()

from collections import defaultdict

# Use defaultdict to efficiently group data by app_id and plan_id
data_by_app_plan = defaultdict(dict)

for pricing_plan_detail_row in pricing_plan_detail_rows:
    app_id = pricing_plan_detail_row[0]
    plan_id = pricing_plan_detail_row[1]
    title = pricing_plan_detail_row[2]  
    price = pricing_plan_detail_row[3]  
    feature = pricing_plan_detail_row[4]
    data = data_by_app_plan.get((app_id,), {'title': title, 'price': price, 'features': []})
    data['features'].append(feature)  # Append feature to existing list
    data_by_app_plan[(app_id,)] = data
    
for app_row in app_rows:
    app_id=app_row[0]
    app_category=[]
    for cat in category_rows:
        if cat[0] == app_id:
            app_category.append(cat[2])
    app_record = {}
    app_record['id'] = app_row[0]
    app_record['url'] = app_row[1]
    app_record['title'] = app_row[2]
    app_record['developer'] = app_row[3]
    app_record['developer_link'] = app_row[4]
    app_record['icon'] = app_row[5]
    app_record['rating'] = app_row[6]
    app_record['reviews_count'] = app_row[7]
    app_record['category'] = app_category  
    app_ref = db.collection('apps').document(app_row[0])  
    batch.set(app_ref, app_record)
    
    # Create or update subcollection document with data
    data = data_by_app_plan.get((app_id,),{})  # Get data for app_id (plan_id is None)

    pricing_plan_ref = app_ref.collection('pricing_plan_details').document(app_id)  # Use app_id as document ID
    batch.set(pricing_plan_ref, data)
 

    for key_benefit_row in key_benefit_rows:
        if app_row[0] == key_benefit_row[0]: 
            key_benefit_record = {}
            unique_id= unique_id = "one"+str(uuid.uuid4())
            key_benefit_record['title'] = key_benefit_row[1]
            key_benefit_record['description'] = key_benefit_row[2]
            key_benefit_ref = app_ref.collection('key_benefits').document(unique_id) 
            batch.set(key_benefit_ref,key_benefit_record) 
               

    for review_row in review_rows:
        if app_row[0] == review_row[0]:
            unique_id = "one"+str(uuid.uuid4())

            review_record = {}
            review_record['author'] = review_row[1]
            review_record['rating'] = review_row[2]
            review_record['posted_at'] = review_row[3]
            review_ref = app_ref.collection('reviews').document(unique_id)
            batch.set(review_ref, review_record)
            
    batch.commit()
print("polulation done")

polulation done


#### 3. Get results queries

### Get a count of the number of documents in each collection and subcollection.

### a. Get a count of the number of documents in collection apps

In [11]:
# Count documents in apps collection
import pandas as pd
from google.cloud import firestore
from datetime import datetime

db = firestore.Client()
apps_ref = db.collection('apps')
docs = apps_ref.get()
num=len(docs)
print("Number of apps document: ",num)

Number of apps document:  3547


### b. Get a count of the number of documents in collection categories

In [19]:
import pandas as pd 
from google.cloud import firestore 
from google.cloud.firestore_v1 import Client

db = firestore.Client(project="daring-pilot-418321")
apps_ref = db.collection('categories') 
docs = apps_ref.get() 
num=len(docs) 
print("Number of categories document: ",num)

Number of categories document:  12


### c. get a count of the number of documents in each subcollection

In [2]:
import pandas as pd
from google.cloud import firestore
df = pd.read_csv('/home/jupyter/shopify/pricing_plan_details.csv', sep=',', header=0, lineterminator='\n')
pricing_plan_detail_rows = df.values.tolist()

from collections import defaultdict

# Use defaultdict to efficiently group data by app_id and plan_id
data_by_app_plan = defaultdict(dict)

for pricing_plan_detail_row in pricing_plan_detail_rows:
    app_id = pricing_plan_detail_row[0]
    plan_id = pricing_plan_detail_row[1] 
    feature = pricing_plan_detail_row[2]
    data = data_by_app_plan.get((app_id,), {'features': []})
    data['features'].append(feature)  # Append feature to existing list
    data_by_app_plan[(app_id,)] = data
total_count = sum(len(v) for v in data_by_app_plan.values())
print("Number of document count for pricing_plan_details:", total_count)


Number of document count for pricing_plan_details: 6275


In [3]:
import pandas as pd
from google.cloud import firestore
from datetime import datetime
df = pd.read_csv('/home/jupyter/shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_num = df.shape[0]
df = pd.read_csv('/home/jupyter/shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
benefits_num = df.shape[0]

print("Number of Key Benefits document: ",benefits_num)
print("Number of Reviews document: ",reviews_num)

Number of Key Benefits document:  9541
Number of Reviews document:  124601


### List the top 10 "Productivity" apps (whose category.title = "Productivity") sorted by rating in descending order. 
Return the id, title, developer, rating and reviews_count for those apps.

In [85]:
import pandas as pd
from google.cloud import firestore

db = firestore.Client()
apps_ref = db.collection('apps')

# Query for "Productivity" apps
query = (apps_ref
         .where('category', 'array_contains', 'Productivity')
         .order_by('rating', direction=firestore.Query.DESCENDING).limit(10))
         
# Execute the query
results = query.stream()

# Process and print the results
for result in results:
    app_data = result.to_dict()
    print(f"App ID: {app_data['id']}")
    print(f"App Title: {app_data['title']}")
    print(f"Developer: {app_data['developer']}")
    print(f"Rating: {app_data['rating']}")
    print(f"Review Count: {app_data['reviews_count']}")
    print("-" * 20)  # Optional separator between apps



App ID: fe5ae45c-379c-42bf-be7d-b5f6e15fc13c
App Title: Customer Tags
Developer: Union Works Apps
Rating: 5.0
Review Count: 21
--------------------
App ID: f99bb1e3-f326-4f10-8901-491652e9809b
App Title: Order Tagger
Developer: Union Works Apps
Rating: 5.0
Review Count: 68
--------------------
App ID: f864e3bd-da0e-41dc-be65-984325331475
App Title: SilkRoad ‑ Facebook Auto Ads
Developer: SilkRoad
Rating: 5.0
Review Count: 2
--------------------
App ID: f5344e64-9cda-4d97-b198-2aaeb5170518
App Title: Xporter Data Export Tool
Developer: Modd Apps Inc.
Rating: 5.0
Review Count: 223
--------------------
App ID: ed77a32d-0fa3-458b-b639-e01ea7b78ec0
App Title: FraudBlock Fraud Prevention
Developer: ShopFox
Rating: 5.0
Review Count: 6
--------------------
App ID: e0e231d6-4988-4a8c-ad84-cf18d0f38738
App Title: Automation Fox
Developer: Automation Fox
Rating: 5.0
Review Count: 1
--------------------
App ID: d75fa395-3a4d-41c3-88d1-2f802c1e2411
App Title: AirPower
Developer: BaseGenius
Rating: 

/opt/conda/lib/python3.10/site-packages/google/cloud/firestore_v1/base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


### List the top 10 apps with the highest number of reviews (based on apps.review_count). 
Return the id, title, developer, rating and reviews_count for those apps. 
Sort the results by reviews_count in descending order


In [56]:
query = db.collection("apps").order_by("reviews_count", direction="DESCENDING").limit(10)
app_query=query.select(["id","title","developer", "rating","reviews_count"])

top_apps=app_query.get()
for app in top_apps:
    app_data = app.to_dict()
    print(f"App ID: {app_data['id']}")
    print(f"App Title: {app_data['title']}")
    print(f"Developer: {app_data['developer']}")
    print(f"Rating: {app_data['rating']}")
    print(f"Review Count: {app_data['reviews_count']}")
    print("-" * 20)  # Optional separator between apps


App ID: d9f142ee-b141-4dc4-9353-173db61d2eb0
App Title: Privy ‑ Exit Pop Ups & Email
Developer: Privy
Rating: 4.7
Review Count: 23078
--------------------
App ID: 78ea0810-c008-4a4e-a82f-de0c790e3286
App Title: Free Shipping Bar
Developer: Hextom
Rating: 4.9
Review Count: 8737
--------------------
App ID: b88488b0-9912-44d3-b736-224c36f09d95
App Title: Sales Pop ‑ Popup Notification
Developer: CartKit
Rating: 4.8
Review Count: 6905
--------------------
App ID: e528a60e-94f8-4e92-80e2-5bc6013b8283
App Title: BEST Currency Converter
Developer: Grizzly Apps
Rating: 4.8
Review Count: 5986
--------------------
App ID: be2640c4-01b5-4d52-9f68-cae8c0734d0d
App Title: Recart FB Messenger Marketing
Developer: Recart
Rating: 4.8
Review Count: 5596
--------------------
App ID: 70bff9e0-4316-4cc6-84ce-92fcd1bc6925
App Title: EU Cookie Bar ‑ Cookie GDPR
Developer: Booster Apps
Rating: 4.7
Review Count: 5259
--------------------
App ID: 171816e2-27d4-4552-a65e-ab44a312fe04
App Title: Sales Pop Maste